In [16]:
# ------------------------------------------
# Imports and setup
# ------------------------------------------

import sys, os
import pandas as pd

#print("Current working directory:", os.getcwd())

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

#Import the class EnvironmentalData
from data_import import EnvironmentalData
env_data=EnvironmentalData()
"""
Functions within the class, with arguments:
    get_met(self, weather_station, weather_elements, weather_time, weather_resolution)
    get_nilu(self, threshold, file_path)
    missing_data(self, df)
    show_zeroes(self, df)
"""

'\nFunctions within the class, with arguments:\n    get_met(self, weather_station, weather_elements, weather_time, weather_resolution)\n    get_nilu(self, threshold, file_path)\n    missing_data(self, df)\n    show_zeroes(self, df)\n'

In [17]:
# ------------------------------------------
# FORMATTING OF DATAFRAME VISUALISATION
# ------------------------------------------

# A function to make pandas show an entire dataframe in a readable manner
def pretty_data(df):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    display(df)

In [18]:
# ------------------------------------------
# WEATHER DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the weather data
weather_station = 'SN68860' # The ID of the weather station to fetch data from, can be multiple stations separated by a comma - 'SN68860' is Voll weatherstation
weather_elements = 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)' # The measurements to include in the dataset, here: temperautre, precipitation, and wind speed
weather_time = '2024-01-30/2024-03-19' # The stretch of time to fetch weather data for
weather_resolution = 'P1D' # The level of granularity in the weather data. P1D is once per day. P1H is once per hour.

# Run the module for importing the weather data, and save it to a dataframe
df_weather = env_data.get_met(weather_station, weather_elements, weather_time, weather_resolution)

# Check if data was fetched successfully and check for missing values
if df_weather is None:
    print("Failed to fetch weather data.")
else:
    env_data.missing_data(df_weather)



# Displaying the weather data as a table
#pretty_data(df_weather)

Data collected from frost.met.no!
There are  245 lines of data in this dataframe.

No missing values found in the weather data! 



In [21]:
# ------------------------------------------
# AIR QUALITY DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the air quality data
threshold = 95 # The threshold below which we will not use the data for analysis (uptime % of day)
file_path = '../Data/luftkvalitet_trondheim_dag.csv' #Choose the file to get air quality data from

# Run the module for importing the air quality data, and save it to a dataframe
df_quality = env_data.get_nilu(threshold, file_path)

# Check if data was fetched successfully and check for missing values
if df_quality is None:
    print("Failed to fetch weather data.")
else:
    env_data.missing_data(df_quality)

no_air_quality_data = env_data.show_zeroes(df_quality)
print('These rows contain data of too poor quality',no_air_quality_data)

# Displaying the air quality data as a table
#pretty_data(df_quality)

Data collected from nilu.com!
There are  367 lines of data in this dataframe.

⚠️ Missing values found at these locations:
Missing value at Row 107, Column 'Elgeseter NO µg/m³ Day'
Missing value at Row 107, Column 'Elgeseter NO2 µg/m³ Day'
Missing value at Row 107, Column 'Elgeseter NOx µg/m³ Day'
Missing value at Row 108, Column 'Elgeseter NO µg/m³ Day'
Missing value at Row 108, Column 'Elgeseter NO2 µg/m³ Day'
Missing value at Row 108, Column 'Elgeseter NOx µg/m³ Day'
Missing value at Row 109, Column 'Elgeseter NO µg/m³ Day'
Missing value at Row 109, Column 'Elgeseter NO2 µg/m³ Day'
Missing value at Row 109, Column 'Elgeseter NOx µg/m³ Day'
Missing value at Row 110, Column 'Elgeseter NO µg/m³ Day'
Missing value at Row 110, Column 'Elgeseter NO2 µg/m³ Day'
Missing value at Row 110, Column 'Elgeseter NOx µg/m³ Day'
Missing value at Row 111, Column 'Elgeseter NO µg/m³ Day'
Missing value at Row 111, Column 'Elgeseter NO2 µg/m³ Day'
Missing value at Row 111, Column 'Elgeseter NOx µg/m³ Da

In [20]:
# *** EVERYTHING below this line is purely for local testing purposes of this specific file ***
'''Fjern # foran de linjene under som kjører en print funksjon, for å se om det virker :)'''

    # Print the first bit of the weather data to confirm the function is working
#print(df_weather.head())

    # Print the index list of the excluded air quality data:
#print('Rows with poor or no coverage can be found at the following rows: \n',exclude_data)

'Fjern # foran de linjene under som kjører en print funksjon, for å se om det virker :)'